In [1]:
import os
import pandas as pd

# import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Load the train , test and validation data and labels
print(os.listdir("../../data/raw/Food"))
labels_df = pd.read_csv("../../data/raw/Food/labels/labels.csv")
# Define the data transformations
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, dataframe, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        # Load image files
        self.image_files = sorted(
            [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f))]
        )
        # Initialize a dictionary to map frame identifiers to labels
        self.labels_map = {}
        # Populate the labels_map
        for _, row in dataframe.iterrows():
            self.labels_map[row["Frame_Number"]] = row["Label"]
        # Filter out image files without a corresponding label
        self.image_files = [
            img
            for img in self.image_files
            if os.path.splitext(img)[0] in self.labels_map
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        full_img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(full_img_path).convert("RGB")

        frame_identifier = os.path.splitext(img_name)[0]
        label = self.labels_map.get(frame_identifier)

        # Handle the unlikely case where a label is not found
        if label is None:
            print(f"Warning: Label not found for image: {img_name}. Skipping...")
            return None  # This should be handled by your dataloader or skipped

        if self.transform:
            image = self.transform(image)

        return image, label


# Update your DataLoader to skip None types (which we use for missing labels)
from torch.utils.data.dataloader import default_collate


def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    return default_collate(batch)


train_data_path = "../../data/raw/Food/train"
test_data_path = "../../data/raw/Food/test"
val_data_path = "../../data/raw/Food/val"

train_dataset = CustomImageDataset(train_data_path, labels_df, transform)
test_dataset = CustomImageDataset(
    test_data_path, labels_df, transform
)  # Adjust these according to actual splits
val_dataset = CustomImageDataset(
    val_data_path, labels_df, transform
)  # Adjust these according to actual splits

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# !pip install efficientnet_pytorch
# !pip install optuna

['val', 'train', 'labels', 'test']


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from efficientnet_pytorch import EfficientNet  # Corrected import for your requirement
import optuna


# EarlyStopping class definition
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                           Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float("inf")

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ..."
            )
        torch.save(model.state_dict(), "checkpoint.pt")
        self.val_loss_min = val_loss


# Check for GPU availability and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models  # Import for ResNet
import optuna


def train_model(trial, train_loader, val_loader, num_epochs=5):
    # Sample hyperparameters from the trial
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 0.1)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop", "Adamax", "Adagrad", "Adadelta"])
    num_epochs = trial.suggest_int("num_epochs", 500, 1500)

    # Initialize and move the model to the specified device
    model = models.resnet50(pretrained=False, num_classes=2).to(device)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Define optimizer based on the sampled name
    optimizer = {
        "Adam": optim.Adam(model.parameters(), lr=learning_rate),
        "SGD": optim.SGD(model.parameters(), lr=learning_rate),
        "RMSprop": optim.RMSprop(model.parameters(), lr=learning_rate),
        "Adamax": optim.Adamax(model.parameters(), lr=learning_rate),
        "Adagrad": optim.Adagrad(model.parameters(), lr=learning_rate),
        "Adadelta": optim.Adadelta(model.parameters(), lr=learning_rate)
    }[optimizer_name]

    # Early stopping initialization
    early_stopping = EarlyStopping(patience=20, verbose=True)

    # Lists to store training and validation losses
    train_losses = []
    valid_losses = []

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)  # Move data to the device

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss for the epoch
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        # Evaluate the model on the validation set
        model.eval()  # Set the model to evaluation mode
        valid_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()

        # Calculate average validation loss for the epoch
        valid_loss /= len(val_loader)
        valid_losses.append(valid_loss)

        # Report intermediate results to Optuna
        trial.report(valid_loss, epoch)

        # Early stopping call
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        # Print training statistics
        print(
            f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}"
        )

    return valid_losses[-1]

# Define the objective function for Optuna
def objective(trial):
    return train_model(trial, train_loader, val_loader)

# Create a study object and optimize hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=17)

# Get the best hyperparameters and train the final model with them
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Save the Optuna study object
import pickle
with open("study.pkl", "wb") as f:
    pickle.dump(study, f)

[I 2024-04-18 09:13:38,234] A new study created in memory with name: no-name-c8fcb079-a7f7-4930-b3d2-7fcd924a1c4c


/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/olarinoyem/miniconda3/envs/deep_tf/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Validation loss decreased (inf --> 0.646943).  Saving model ...
Epoch [1/1376], Train Loss: 0.6443, Valid Loss: 0.6469
Validation loss decreased (0.646943 --> 0.508634).  Saving model ...
Epoch [2/1376], Train Loss: 0.5500, Valid Loss: 0.5086
EarlyStopping counter: 1 out of 20
Epoch [3/1376], Train Loss: 0.4088, Valid Loss: 0.7805
EarlyStopping counter: 2 out of 20
Epoch [4/1376], Train Loss: 0.2935, Valid Loss: 0.9254
Validation loss decreased (0.508634 --> 0.308947).  Saving model ...
Epoch [5/1376], Train Loss: 0.2683, Valid Loss: 0.3089
Validation loss decreased (0.308947 --> 0.243313).  Saving model ...
Epoch [6/1376], Train Loss: 0.1932, Valid Loss: 0.2433
EarlyStopping counter: 1 out of 20
Epoch [7/1376], Train Loss: 0.1796, Valid Loss: 1.1451
EarlyStopping counter: 2 out of 20
Epoch [8/1376], Train Loss: 0.2005, Valid Loss: 2.3867
EarlyStopping counter: 3 out of 20
Epoch [9/1376], Train Loss: 0.1424, Valid Loss: 0.4903
EarlyStopping counter: 4 out of 20
Epoch [10/1376], Train L

[I 2024-04-18 09:19:41,771] Trial 0 finished with value: 0.37995157356954223 and parameters: {'learning_rate': 0.030210944646979616, 'optimizer': 'Adadelta', 'num_epochs': 1376}. Best is trial 0 with value: 0.37995157356954223.


EarlyStopping counter: 20 out of 20
Early stopping
Validation loss decreased (inf --> 2.468570).  Saving model ...
Epoch [1/1010], Train Loss: 6.7550, Valid Loss: 2.4686
Validation loss decreased (2.468570 --> 0.621739).  Saving model ...
Epoch [2/1010], Train Loss: 0.7966, Valid Loss: 0.6217
Validation loss decreased (0.621739 --> 0.495134).  Saving model ...
Epoch [3/1010], Train Loss: 0.5646, Valid Loss: 0.4951
Validation loss decreased (0.495134 --> 0.464866).  Saving model ...
Epoch [4/1010], Train Loss: 0.4979, Valid Loss: 0.4649
EarlyStopping counter: 1 out of 20
Epoch [5/1010], Train Loss: 0.4507, Valid Loss: 0.5053
EarlyStopping counter: 2 out of 20
Epoch [6/1010], Train Loss: 0.4090, Valid Loss: 0.5154
Validation loss decreased (0.464866 --> 0.364369).  Saving model ...
Epoch [7/1010], Train Loss: 0.3886, Valid Loss: 0.3644
Validation loss decreased (0.364369 --> 0.327312).  Saving model ...
Epoch [8/1010], Train Loss: 0.3786, Valid Loss: 0.3273
EarlyStopping counter: 1 out o

...

EarlyStopping counter: 20 out of 20
Early stopping
Validation loss decreased (inf --> 10.482030).  Saving model ...
Epoch [1/609], Train Loss: 5.2282, Valid Loss: 10.4820
Validation loss decreased (10.482030 --> 0.608061).  Saving model ...
Epoch [2/609], Train Loss: 0.8771, Valid Loss: 0.6081
EarlyStopping counter: 1 out of 20
Epoch [3/609], Train Loss: 0.5704, Valid Loss: 0.7786
EarlyStopping counter: 2 out of 20
Epoch [4/609], Train Loss: 0.5145, Valid Loss: 0.6532
EarlyStopping counter: 3 out of 20
Epoch [5/609], Train Loss: 0.4606, Valid Loss: 0.6391
Validation loss decreased (0.608061 --> 0.553202).  Saving model ...
Epoch [6/609], Train Loss: 0.5255, Valid Loss: 0.5532
Validation loss decreased (0.553202 --> 0.465825).  Saving model ...
Epoch [7/609], Train Loss: 0.4599, Valid Loss: 0.4658
EarlyStopping counter: 1 out of 20
Epoch [8/609], Train Loss: 0.4183, Valid Loss: 0.5071
Validation loss decreased (0.465825 --> 0.419112).  Saving model ...
Epoch [9/609], Train Loss: 0.3645,

[I 2024-04-18 10:23:02,905] Trial 7 finished with value: 0.14465774438576773 and parameters: {'learning_rate': 0.05831790061428621, 'optimizer': 'Adamax', 'num_epochs': 609}. Best is trial 6 with value: 0.05414052632355985.


EarlyStopping counter: 20 out of 20
Early stopping
Validation loss decreased (inf --> 0.756577).  Saving model ...
Epoch [1/733], Train Loss: 1.3821, Valid Loss: 0.7566
Validation loss decreased (0.756577 --> 0.701107).  Saving model ...
Epoch [2/733], Train Loss: 0.7930, Valid Loss: 0.7011
Validation loss decreased (0.701107 --> 0.507159).  Saving model ...
Epoch [3/733], Train Loss: 0.6091, Valid Loss: 0.5072
EarlyStopping counter: 1 out of 20
Epoch [4/733], Train Loss: 0.4886, Valid Loss: 1.2558
Validation loss decreased (0.507159 --> 0.392743).  Saving model ...
Epoch [5/733], Train Loss: 0.4336, Valid Loss: 0.3927
EarlyStopping counter: 1 out of 20
Epoch [6/733], Train Loss: 0.4943, Valid Loss: 0.4850
EarlyStopping counter: 2 out of 20
Epoch [7/733], Train Loss: 0.3871, Valid Loss: 0.5486
EarlyStopping counter: 3 out of 20
Epoch [8/733], Train Loss: 0.3893, Valid Loss: 0.4039
EarlyStopping counter: 4 out of 20
Epoch [9/733], Train Loss: 0.3600, Valid Loss: 0.6300
EarlyStopping cou

[I 2024-04-18 10:34:45,184] Trial 8 finished with value: 0.11468920242198995 and parameters: {'learning_rate': 0.009506452210691218, 'optimizer': 'Adam', 'num_epochs': 733}. Best is trial 6 with value: 0.05414052632355985.


EarlyStopping counter: 20 out of 20
Early stopping
Validation loss decreased (inf --> 6336035.031250).  Saving model ...
Epoch [1/644], Train Loss: 4.6450, Valid Loss: 6336035.0312
Validation loss decreased (6336035.031250 --> 1.305996).  Saving model ...
Epoch [2/644], Train Loss: 0.7762, Valid Loss: 1.3060
Validation loss decreased (1.305996 --> 0.630073).  Saving model ...
Epoch [3/644], Train Loss: 0.6790, Valid Loss: 0.6301
EarlyStopping counter: 1 out of 20
Epoch [4/644], Train Loss: 0.6650, Valid Loss: 0.6461
Validation loss decreased (0.630073 --> 0.610283).  Saving model ...
Epoch [5/644], Train Loss: 0.6165, Valid Loss: 0.6103
Validation loss decreased (0.610283 --> 0.597445).  Saving model ...
Epoch [6/644], Train Loss: 0.5997, Valid Loss: 0.5974
EarlyStopping counter: 1 out of 20
Epoch [7/644], Train Loss: 0.5620, Valid Loss: 0.6390


In [ ]:
import pickle

# Assuming 'study' is your Optuna study object
with open("study2.pkl", "wb") as f:
    pickle.dump(study, f)

NameError: name 'study' is not defined